# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902028


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:22,  2.85s/it]

Rendering models:  10%|▉         | 3/31 [00:06<00:58,  2.10s/it]

Rendering models:  13%|█▎        | 4/31 [00:08<00:57,  2.14s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:42,  1.64s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:17<00:01,  1.95it/s]

Rendering models:  94%|█████████▎| 29/31 [00:18<00:01,  1.97it/s]

Rendering models:  97%|█████████▋| 30/31 [00:18<00:00,  2.09it/s]

Rendering models: 100%|██████████| 31/31 [00:19<00:00,  2.25it/s]

Jes8x                                      0.000361
not-logged-in-a924460f0f1f97422ce6         0.010107
eatyourgreens                              0.000756
equidad1                                   0.064766
xperry9702                                 0.000736
mollollo                                   0.000168
I_wish_I_can_study_at_Starfleet_Academy    0.000908
Drew_Farnsworth                            0.001063
Thedunkmasta                               0.003033
brodicius                                  0.001244
not-logged-in-14849be8c56260e26d09         0.001359
elsawinslow                                0.005829
not-logged-in-4d8ad53eb5485d8c8f8c         0.000162
raisamaduro                                0.006884
Adrian_P                                   0.001658
jmartinez468                               0.003027
not-logged-in-314e7eb9cecafaf63009         0.000243
not-logged-in-3ae935e3afaee843f530         0.011005
aisha3540                                  0.001926
crush202020 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())